In [1]:
import json
import warnings
import requests
import tqdm

In [2]:
#Load your api key
with open('../api_keys.json') as f:
    api_keys = json.load(f)
alt_metric_key = api_keys['alt_metric']

def get_reduced_alt_values(r):
    """when some info is missing extracts less information from the response object"""
    score = r['score']
    missing_values = False
    try:
        percentile = r['altmetric_score']['context_for_score']['all']['percentile']
    except:
        percentile = float('NaN')
        missing_values = True
    try:
        similar_age_3m_percentile = r['altmetric_score']['context_for_score']['similar_age_3m']
    except:
        similar_age_3m_percentile = float('NaN')
        missing_values = True
    try:
        total_readers = 0
        for key in r['counts']['readers'].keys():
            total_readers += int(r['counts']['readers'][key])
    except:
        total_readers = float('NaN')
        missing_values = True
    alt_values = {'score': score}, {'percentile': percentile}, {'similar_age_3m_percentile': similar_age_3m_percentile}, {'total_readers': total_readers}
    return alt_values, missing_values
def altmetric_arxiv_requests(arxiv):
    """make request with arxiv id"""
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/arxiv_id/{arxiv}?key={alt_metric_key}')
    if r.status_code != 200:
        print('Warning - Response Code ', r.status_code)
    return r.json()
def altmetric_doi_requests(doi):
    """make request with doi"""
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/doi/{doi}?key={alt_metric_key}')
    if r.status_code != 200:
        print('Warning - Response Code ', r.status_code, 'DOI', doi)
    return r.json()
altmetric_doi_requests('10.1177%2F0278364913491297')['altmetric_score']['context_for_score']['all']

{'total_number_of_other_articles': 22796179,
 'mean': 10.19502521226795,
 'rank': 2415136,
 'this_scored_higher_than_pct': 89,
 'this_scored_higher_than': 20369550,
 'rank_type': 'exact',
 'sample_size': 22796179,
 'percentile': 89}

In [3]:
suffix = '' #'_04_01_2023'
file_name = f'../data/data_sorted_only_w_ids{suffix}.json'
with open(file_name, "r") as ds:
    retrieved_data = json.load(ds)
print('Number of Datasets:', len(retrieved_data))
c = 0
c_found = 0
c_data_added = 0
c_found_doi = 0
c_data_added_doi = 0
c_no_missing_values = 0
c_updated_altmetrics = 0
for idx_paper, paper in enumerate(retrieved_data):
    c += 1
    item = [] # so request won't work unless there is a new item
    no_alt_values = False
    related_paper = False
    arxiv_value = False # check if we have link for axiv in paper['relatedPaper']
    arxiv_request_worked = False #check if request worked
    arxiv_request_extracted = False

    try:
        item = paper['relatedPaper']
        related_paper = True
    except Exception as exec:
        print(exec)
        print('No related paper', paper['id'])
        break
    if related_paper:
        # --- try with arxiv
        if item.find("https://arxiv.org/abs") != -1:
            #print("arXiv Article URL")
            arxiv_id = item[22:]
            arxiv_value = True
        elif item.find("https://arxiv.org/pdf") != -1:
            #print("arXiv PDF URL")
            arxiv_id = item[22:-4]
            arxiv_value = True
        elif item.find("arxiv") != -1:
            print("Broken arXiv URL")
        else:
            print("No arXiv URL")
        if arxiv_value:
            try:
                r = altmetric_arxiv_requests(arxiv_id)
                arxiv_request_worked = True
                c_found += 1
            except Exception as exec:
                print('Arxiv request returned something wrong', exec)
            if arxiv_request_worked:
                try:
                    alt_values_arxiv, missing_value_arxiv = get_reduced_alt_values(r)
                    arxiv_request_extracted = True
                    c_data_added += 1
                except Exception as exec:
                    print('Arxiv request returned something wrong', exec)

    # --- try with DOI
    has_doi = False
    doi_request_worked = False #check if request worked
    doi_request_extracted = False
    try:
        doi = paper['DOI']
        has_doi = True
    except Exception as exec:
        print(exec)
        print('No DOI for paper', paper['id'])
    if has_doi:
        try:
            r = altmetric_doi_requests(doi)
            doi_request_worked = True
            c_found_doi += 1
        except Exception as exec:
            print('DOI request returned something wrong', exec)
        if doi_request_worked:
            try:
                alt_values_doi, missing_value_doi = get_reduced_alt_values(r)
                doi_request_extracted = True
                c_data_added_doi += 1
            except Exception as exec:
                print('Arxiv request returned something wrong', exec)

    #decide which result to use if multiple
    if arxiv_request_extracted and doi_request_extracted:
        if missing_value_arxiv:
            alt_values = alt_values_doi
        elif missing_value_doi:
            alt_values = alt_values_arxiv
        elif alt_values_doi[0]['score'] > alt_values_arxiv[0]['score']:
            alt_values = alt_values_doi
        else:
            alt_values = alt_values_arxiv

    # counting complete entries
    if (arxiv_request_extracted and not missing_value_arxiv) or (doi_request_extracted and not missing_value_doi):
        c_no_missing_values += 1
    #if only one worked, select the one that worked
    if doi_request_extracted and not arxiv_request_extracted:
        alt_values = alt_values_doi

    if arxiv_request_extracted and not doi_request_extracted:
        alt_values = alt_values_arxiv

    if not arxiv_request_extracted and not doi_request_extracted:
        continue
    c_updated_altmetrics += 1
    retrieved_data[idx_paper].update({'altmetrics': alt_values})
with open(f'../data/data_sorted_only_w_ids{suffix}_altmetrics.json', 'w', encoding='utf-8') as f:
    json.dump(retrieved_data, f)
print('found arxiv:', c_found, '/', c)
print('added arxiv:', c_data_added, '/', c)
print('found doi:', c_found_doi, '/', c)
print('added doi:', c_data_added_doi, '/', c)
print('updated altmetrics in data:', c_updated_altmetrics, '/', c)

Number of Datasets: 206
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
Warning - Response Code  404 DOI 10.1109/IJCNN.2013.6706807
DOI request returned something wrong Expecting value: line 1 column 1 (char 0)
No arXiv URL
No arXiv URL
No arXiv URL
Warning - Response Code  404 DOI 10.1177/0278364913507326
DOI request returned something wrong Expecting value: line 1 column 1 (char 0)
Warning - Response Code  404 DOI -
DOI request returned something wrong Expecting value: line 1 column 1 (char 0)
Warning - Response Code  404
Arxiv request returned something wrong Expecting value: line 1 column 1 (char 0)
Warning - Response Code  404 DOI 10.48550/arXiv.1608.01230
DOI request returned something wrong Expecting value: line 1 column 1 (char 0)
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
No arXiv URL
Warning - Response Code  404 DOI 10.48550/arXiv.2006.14480
DOI request returned something wrong Expecting value: line 1 column 1 (